# Datast 5 : breastCancer

### Importing the libraries 

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import time
import psutil
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
from sklearn.datasets import load_breast_cancer

### Importinng the PlumberOptimizer

In [3]:
import sys
sys.path.append('../')
from Plumber import PlumberOptimizer

### Loading the Data and performing feature engineering:

In [4]:
# Load Breast Cancer dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Optional: Split for holdout evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Split the data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (455, 30)
Test set shape: (114, 30)


### Testing the performance and efficiency of the PlumberOptimizer

In [5]:
# Arrays to store execution times, memory usage, CPU usage, and evaluatin scores
plumber_time = np.zeros(5,'float')
plumber_mem = np.zeros(5,'float')
plumber_cpu =  np.zeros(5,'float')
plumber_score = np.zeros(5,'float')
for i in range(5):
    opt = PlumberOptimizer(X_train, y_train)

    # Monitor the current process, record the start time, initial CPU usage, and memory usage in MB
    process = psutil.Process(os.getpid())
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024) 


    best = opt.optimize()

    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  

    # Calculating execution time, memory usage, and CPU usage
    plumber_time[i] = end_time - start_time
    plumber_mem[i] = end_mem - start_mem
    plumber_cpu[i] = end_cpu - start_cpu

    # Training and evaluating the model using the best parameters
    rf = RandomForestClassifier(**best , random_state=42 )
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)


    plumber_score[i] = accuracy_score(y_test, y_pred)

# summary for the Plumber Optimizer
print("\n" *3)
print("*************")
print(f"Execution Time: mean={np.mean(plumber_time):.2f} seconds, std={np.std(plumber_time)} seconds, max={np.max(plumber_time)} seconds, min={np.min(plumber_time)} seconds")
print(f"CPU Usage: mean={np.mean(plumber_cpu):.2f}%, std={np.std(plumber_cpu):.2f}%, max={np.max(plumber_cpu):.2f}%, min={np.min(plumber_cpu):.2f}%")
print(f"Memory Usage: mean={np.mean(plumber_mem):.2f} MB, std={np.std(plumber_mem):.2f} MB, max={np.max(plumber_mem):.2f} MB, min={np.min(plumber_mem):.2f} MB")
print(f"Validation Score: mean={np.mean(plumber_score)} , std={np.std(plumber_score)} , max={np.max(plumber_score)} , min={np.min(plumber_score)} ")

A single model takes  0.07993388175964355 seconds to run
Suggested number of trails is 150 
Best Parameters: {'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best Score: 0.960439177413733
A single model takes  0.06674408912658691 seconds to run
Suggested number of trails is 150 
Best Parameters: {'n_estimators': 50, 'max_depth': 11, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best Score: 0.960439177413733
A single model takes  0.06676101684570312 seconds to run
Suggested number of trails is 150 
Best Parameters: {'n_estimators': 200, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best Score: 0.9626466829324968
A single model takes  0.06832599639892578 seconds to run
Suggested number of trails is 150 
Best Parameters: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best Score: 0.9626466829324968
A si

### Testing the performance and efficiency of the GridSearch

In [6]:
# Defining GridSearch parameters
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  
    'max_features': ['sqrt', 'log2'],  
    'max_depth': [10, 20, 30, 40, 50],  
    'min_samples_split': [2, 5, 10, 12, 16, 20], 
    'min_samples_leaf': [2, 5, 6, 8,12,16, 20]  
}

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')

# Monitor the current process, record the start time, initial CPU usage, and memory usage in MB
process = psutil.Process(os.getpid())
start_time = time.time()
start_cpu = process.cpu_percent(interval=None)
start_mem = process.memory_info().rss / (1024 * 1024)  

grid_search.fit(X_train, y_train)

end_time = time.time()
end_cpu = process.cpu_percent(interval=None)
end_mem = process.memory_info().rss / (1024 * 1024) 

# Print metrics for GridSearch
print(f"Execution Time: {end_time - start_time:.2f} seconds")
print(f"CPU Usage: {end_cpu - start_cpu:.2f}%")
print(f"Memory Usage: {end_mem - start_mem:.2f} MB")


# Training the model with the best parameters and evaluate
rf = RandomForestClassifier(** grid_search.best_params_, random_state=42 )
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Validation Accuracy: {accuracy}")
print(grid_search.best_params_)




Execution Time: 131.57 seconds
CPU Usage: 3.20%
Memory Usage: -9.27 MB
Validation Accuracy: 0.956140350877193
{'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


### Testing the performance and efficiency of the RandomSearch

In [7]:
# Defining the parameter for RandomizedSearch
param_dist = {
    'n_estimators': np.arange(50, 500),
    'max_features': ['sqrt', 'log2'],
    'max_depth': np.arange(10, 31),
    'min_samples_split': np.arange(2, 50),
    'min_samples_leaf': np.arange(1, 50)
}
# Arrays to store execution times, memory usage, CPU usage, and evaluation scores
rand_time = np.zeros(5,'float')
rand_mem = np.zeros(5,'float')
rand_cpu =  np.zeros(5,'float')
rand_score = np.zeros(5,'float')
for i in range(5):
    rf = RandomForestClassifier(random_state=42)
    random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                    n_iter=250, cv=3, n_jobs=-1)
    
    # Monitor the current process, record the start time, initial CPU usage, and memory usage in MB
    process = psutil.Process(os.getpid())
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024)  

    random_search.fit(X_train, y_train)

    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  

    # Calculating execution time, memory usage, and CPU usage
    rand_time[i] = end_time - start_time
    rand_mem[i] = end_mem - start_mem
    rand_cpu[i] = end_cpu - start_cpu

    # Training the model with the best parameters and evaluate
    rf = RandomForestClassifier(** random_search.best_params_ , random_state=42 )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)


    rand_score[i] = accuracy_score(y_test, y_pred)

#RandomizedSearchCV results
print(f"Execution Time: mean={np.mean(rand_time):.2f} seconds, std={np.std(rand_time)} seconds, max={np.max(rand_time)} seconds, min={np.min(rand_time)} seconds")
print(f"CPU Usage: mean={np.mean(rand_cpu):.2f}%, std={np.std(rand_cpu):.2f}%, max={np.max(rand_cpu):.2f}%, min={np.min(rand_cpu):.2f}%")
print(f"Memory Usage: mean={np.mean(rand_mem):.2f} MB, std={np.std(rand_mem):.2f} MB, max={np.max(rand_mem):.2f} MB, min={np.min(rand_mem):.2f} MB")
print(f"Validation Score: mean={np.mean(rand_score)} , std={np.std(rand_score)} , max={np.max(rand_score)} , min={np.min(rand_score)} ")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, c

Execution Time: mean=14.21 seconds, std=0.45044322324710945 seconds, max=15.09384822845459 seconds, min=13.823386192321777 seconds
CPU Usage: mean=5.06%, std=0.51%, max=5.60%, min=4.20%
Memory Usage: mean=0.43 MB, std=0.84 MB, max=2.11 MB, min=0.00 MB
Validation Score: mean=0.9543859649122807 , std=0.0035087719298245723 , max=0.956140350877193 , min=0.9473684210526315 


### Testing the performance and efficiency of the BayesSearchCV

In [8]:
# Define the parameter for BayesSearchCV
param_dist = {
    'n_estimators': Integer(50, 500),
    'max_features': Categorical(['sqrt', 'log2']),
    'max_depth': Integer(10, 50),
    'min_samples_split': Integer(2, 50),
    'min_samples_leaf': Integer(1, 50)
}
# Arrays to store execution times, memory usage, CPU usage, and evaluation scores
rand_time = np.zeros(5,'float')
rand_mem = np.zeros(5,'float')
rand_cpu =  np.zeros(5,'float')
rand_score = np.zeros(5,'float')
for i in range(5):
    print(i)
    rf = RandomForestClassifier(random_state=42)
    bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_dist,
                             n_iter=70, cv=3, n_jobs=-1)

    # Monitor the current process, record the start time, initial CPU usage, and memory usage in MB
    process = psutil.Process(os.getpid())
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024) 

    bayes_search.fit(X_train, y_train)

    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  

    # Calculating execution time, memory usage, and CPU usage
    rand_time[i] = end_time - start_time
    rand_mem[i] = end_mem - start_mem
    rand_cpu[i] = end_cpu - start_cpu

    # Training the model with the best parameters and evaluate
    rf = RandomForestClassifier(** bayes_search.best_params_ , random_state=42 )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)


    rand_score[i] = accuracy_score(y_test, y_pred)


# BayesSearchCV results
print(f"Execution Time: mean={np.mean(rand_time):.2f} seconds, std={np.std(rand_time)} seconds, max={np.max(rand_time)} seconds, min={np.min(rand_time)} seconds")
print(f"CPU Usage: mean={np.mean(rand_cpu):.2f}%, std={np.std(rand_cpu):.2f}%, max={np.max(rand_cpu):.2f}%, min={np.min(rand_cpu):.2f}%")
print(f"Memory Usage: mean={np.mean(rand_mem):.2f} MB, std={np.std(rand_mem):.2f} MB, max={np.max(rand_mem):.2f} MB, min={np.min(rand_mem):.2f} MB")
print(f"Validation Score: mean={np.mean(rand_score)} , std={np.std(rand_score)} , max={np.max(rand_score)} , min={np.min(rand_score)} ")


0
1
2
3
4
Execution Time: mean=100.02 seconds, std=4.588859426553743 seconds, max=104.7781810760498 seconds, min=93.81323409080505 seconds
CPU Usage: mean=1010.76%, std=3.82%, max=1016.40%, min=1004.60%
Memory Usage: mean=75.17 MB, std=137.24 MB, max=349.53 MB, min=1.17 MB
Validation Score: mean=0.9526315789473683 , std=0.004297350425935414 , max=0.956140350877193 , min=0.9473684210526315 
